# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 
### Ομάδα 44
* Βασιλάκος Μιχαήλ <br>
Α.Μ.: 03117069
* Γαβαλάς Παντελεήμων <br>
Α.Μ.: 03117075

## Εγκατάσταση πακέτων και βιβλιοθηκών

In [1]:
!pip install --upgrade nltk
!pip install --upgrade joblib

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk
import string
from tqdm.notebook import tqdm
import pickle
import re

In [3]:
clean_run = False

## Εισαγωγή του Dataset

Το σύνολο δεδομένων με το οποίο θα δουλέψουμε είναι βασισμένο στο [Carnegie Mellon Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). Πρόκειται για ένα dataset με 22.301 περιγραφές ταινιών. Η περιγραφή κάθε ταινίας αποτελείται από τον τίτλο της, μια ή περισσότερες ετικέτες που χαρακτηρίζουν το είδος της ταινίας και τέλος τη σύνοψη της υπόθεσής της. Αρχικά εισάγουμε το dataset (χρησιμοποιήστε αυτούσιο τον κώδικα, δεν χρειάζεστε το αρχείο csv) στο dataframe `df_data_1`: 

In [4]:
dataset_url = "https://drive.google.com/uc?export=download&id=1-tN5rC1QCgwqL67cvIXdMbtd-Xf9Xqow"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Κάθε ομάδα θα δουλέψει σε **ένα μοναδικό υποσύνολο 5.000 ταινιών** (διαφορετικό dataset για κάθε ομάδα) ως εξής:

1. Κάθε ομάδα του εργαστηρίου νευρωνικών έχει έναν αριθμό στο helios. Θα βάλετε τον αριθμό αυτό στη μεταβλητή team_seed_number στο επόμενο κελί κώδικα.

2. Το data frame `df_data_2` έχει γραμμές όσες και οι ομάδες και 5.000 στήλες. Σε κάθε ομάδα αντιστοιχεί η γραμμή του πίνακα με το `team_seed_number` της. Η γραμμή αυτή θα περιλαμβάνει 5.000 διαφορετικούς αριθμούς που αντιστοιχούν σε ταινίες του αρχικού dataset. 

3. Τρέξτε τον κώδικα. Θα προκύψουν τα μοναδικά για κάθε ομάδα  titles, categories, catbins, summaries και corpus με τα οποία θα δουλέψετε.

In [5]:
# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 44

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1-oeMf7BFmiH70XObgC9WOTo-a9cDJZwP"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus

- Ο πίνακας **titles** περιέχει τους τίτλους των ταινιών. Παράδειγμα: 'Sid and Nancy'.
- O πίνακας **categories** περιέχει τις κατηγορίες (είδη) της ταινίας υπό τη μορφή string. Παράδειγμα: '"Tragedy",  "Indie",  "Punk rock",  "Addiction Drama",  "Cult",  "Musical",  "Drama",  "Biopic \[feature\]",  "Romantic drama",  "Romance Film",  "Biographical film"'. Παρατηρούμε ότι είναι μια comma separated λίστα strings, με κάθε string να είναι μια κατηγορία.
- Ο πίνακας **catbins** περιλαμβάνει πάλι τις κατηγορίες των ταινιών αλλά σε δυαδική μορφή ([one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)). Έχει διαστάσεις 5.000 x 322 (όσες οι διαφορετικές κατηγορίες). Αν η ταινία ανήκει στο συγκεκριμένο είδος η αντίστοιχη στήλη παίρνει την τιμή 1, αλλιώς παίρνει την τιμή 0.
- Ο πίνακας **summaries** και η λίστα **corpus** περιλαμβάνουν τις συνόψεις των ταινιών (η corpus είναι απλά ο summaries σε μορφή λίστας). Κάθε σύνοψη είναι ένα (συνήθως μεγάλο) string. Παράδειγμα: *'The film is based on the real story of a Soviet Internal Troops soldier who killed his entire unit  as a result of Dedovschina. The plot unfolds mostly on board of the prisoner transport rail car guarded by a unit of paramilitary conscripts.'*
- το dataframe corpus_df που είναι απλά το corpus σε μορφή dataframe. Τα summaries βρίσκονται στην κολόνα 0. Πιθανώς να σας βολεύει να κάνετε κάποιες προεπεξεργασίες με dataframes.


Θεωρούμε ως **ID** της κάθε ταινίας τον αριθμό γραμμής της ή το αντίστοιχο στοιχείο της λίστας. Παράδειγμα: για να τυπώσουμε τη σύνοψη της ταινίας με `ID=999` (την χιλιοστή) θα γράψουμε `print(corpus[999])`.

In [6]:
ID = 999
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

['Death Bell 2: Bloody Camp']
['"Thriller",  "Horror"']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0

# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

Η πρώτη εφαρμογή που θα αναπτύξετε θα είναι ένα [σύστημα συστάσεων](https://en.wikipedia.org/wiki/Recommender_system) ταινιών βασισμένο στο περιεχόμενο (content based recommender system). Τα συστήματα συστάσεων στοχεύουν στο να προτείνουν αυτόματα στο χρήστη αντικείμενα από μια συλλογή τα οποία ιδανικά θέλουμε να βρει ενδιαφέροντα ο χρήστης. Η κατηγοριοποίηση των συστημάτων συστάσεων βασίζεται στο πώς γίνεται η επιλογή (filtering) των συστηνόμενων αντικειμένων. Οι δύο κύριες κατηγορίες είναι η συνεργατική διήθηση (collaborative filtering) όπου το σύστημα προτείνει στο χρήστη αντικείμενα που έχουν αξιολογηθεί θετικά από χρήστες που έχουν παρόμοιο με αυτόν ιστορικό αξιολογήσεων και η διήθηση με βάση το περιεχόμενο (content based filtering), όπου προτείνονται στο χρήστη αντικείμενα με παρόμοιο περιεχόμενο (με βάση κάποια χαρακτηριστικά) με αυτά που έχει προηγουμένως αξιολογήσει θετικά.

Το σύστημα συστάσεων που θα αναπτύξετε θα βασίζεται στο **περιεχόμενο** και συγκεκριμένα στις συνόψεις των ταινιών (corpus). 


## Προεπεξεργασία

Το πρώτο βήμα στην επεξεργασία μας είναι ο καθαρισμός των περιγραφών των ταινιών. 

Εκτυπώστε (αρκετές) διαφορετικές περιγραφές ταινιών για να δείτε πιθανά προβλήματα που θα πρέπει να αντιμετωπιστούν.

Τα (ελάχιστα) βήματα καθαρισμού που προτείνουμε είναι:
- μετατροπή όλων των χαρακτήρων σε πεζά,
- αφαίρεση των stopwords. Εδώ σημειώστε ότι για το δεδομένο task του συστήματος συστάσεων που είναι η πρόταση ταινιών ίσως θα είχαν ενδιαφέρον και λίστες stopwords πέραν αυτών της κοινής γλώσσας.
- αφαίρεση σημείων στίξης και ειδικών χαρακτρήρων (special characters). Αυτό δεν γίνεται μόνο με την punkt του NLTK. Θα μπορούσατε να βασιστείτε σε κανονικές εκφράσεις (regular expressions), και
- αφαίρεση πολυ σύντομων συμβολοσειρών.

Προσοχή: το corpus και τα τελικά tokens που θα το αποτελούν θα χρησιμοποιηθούν στη συνέχεια ως κλειδιά για να βρούμε εμφυτεύματα. Για το λόγο αυτό, πρέπει να είστε προσεκτικοί ως προς την εφαρμογή μεθόδων κανονικοποίησης (text normalization) όπως το stemming και το lemmatization.

Αρχικά θα φροντίσουμε όλες οι λέξεις να αποτελούνται από πεζά γράμματα.

In [7]:
summaries_lower = []
for summary in summaries:
  summaries_lower.append(summary[0].lower())

Στη συνέχεια θα εμφανίσουμε τις 10 πρώτες περιγραφές για να έχουμε μια εικόνα της αρχικής κατάστασης στην οποία βρίσκονται.

In [8]:
for i in range(10):
  print(summaries_lower[i])

vivacious lady is a story of love at first sight between a young botany professor named peter morgan jr.  and a nightclub singer named francey . the film also has comedic elements, including repeatedly frustrated attempts by the newlywed couple to find a moment alone with each other. the story begins when peter is sent to manhattan to retrieve his playboy cousin keith ([[james ellison  and immediately falls in love with francey. after a whirlwind one-day courtship, peter and francey get married, and they and keith return to the morgan family's home, where peter teaches at the university run by his father peter morgan sr. . mr. morgan is known for being a proud, overbearing man, so peter is afraid to tell him about the marriage. when they arrive, mr. morgan and peter's high-society fiancée helen  initially take francey for another of keith's girlfriends. while peter decides how to approach his father with the news, francey stays at a women-only hotel, and peter and keith introduce her a

Έπειτα θα χωρίσουμε τις περιγραφές σε λέξεις και θα φιλτράρουμε τα stopwords που περιέχει η nltk μαζί με τα σημεία στίξης.

In [9]:
from nltk import word_tokenize
from nltk.corpus import stopwords
# needed for word_tokenize and filtering
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
if clean_run:
  summaries_words = []
  for summary in tqdm(summaries_lower):
    tokenized = word_tokenize(summary)
    filtered_words = [word for word in tokenized if word not in stopwords.words('english') + list(string.punctuation)]
    summaries_words.append(filtered_words)
    
  with open('summaries_tokenized.dump', 'wb') as fid:
      pickle.dump(summaries_words, fid)
else:
  with open('summaries_tokenized.dump', 'rb') as fid:
    summaries_words = pickle.load(fid)

In [11]:
for i in range(10):
  print(summaries_words[i])

['vivacious', 'lady', 'story', 'love', 'first', 'sight', 'young', 'botany', 'professor', 'named', 'peter', 'morgan', 'jr.', 'nightclub', 'singer', 'named', 'francey', 'film', 'also', 'comedic', 'elements', 'including', 'repeatedly', 'frustrated', 'attempts', 'newlywed', 'couple', 'find', 'moment', 'alone', 'story', 'begins', 'peter', 'sent', 'manhattan', 'retrieve', 'playboy', 'cousin', 'keith', 'james', 'ellison', 'immediately', 'falls', 'love', 'francey', 'whirlwind', 'one-day', 'courtship', 'peter', 'francey', 'get', 'married', 'keith', 'return', 'morgan', 'family', "'s", 'home', 'peter', 'teaches', 'university', 'run', 'father', 'peter', 'morgan', 'sr.', 'mr.', 'morgan', 'known', 'proud', 'overbearing', 'man', 'peter', 'afraid', 'tell', 'marriage', 'arrive', 'mr.', 'morgan', 'peter', "'s", 'high-society', 'fiancée', 'helen', 'initially', 'take', 'francey', 'another', 'keith', "'s", 'girlfriends', 'peter', 'decides', 'approach', 'father', 'news', 'francey', 'stays', 'women-only', 'h

Παρατηρούμε πως εξαιτίας του είδους των προτάσεων που επεξεργαζόμαστε υπάρχουν πολλές λέξεις που επαναλαμβάνονται στις περιγραφές και δεν προσδίδουν σημασία, όπως για παράδειγμα οι λέξεις film, jr., sr., mr., mrs. ή αντικείμενα που δεν είναι λέξεις αλλά αναγνωρίστηκαν από την επεξεργασία μας ως τέτοιες όπως π.χ. ``, '', 's, n't, ’ ή ακόμα και links όπως π.χ. www.veeratamilan.com. Τέλος, αφαιρούμε και πολύ σύντομες συμβολοσειρές (μικρότερες από 3 γράμματα).

In [12]:
extra_filtering = ["film","jr.", "sr.", "mr.", "mrs.", "``", "''", "'s", "n't","’"]
summaries_final = []
for summary in tqdm(summaries_words):
  filtered_words = [word for word in summary if (word not in extra_filtering and 
                                                 len(word)>2 and not re.search("^www\.[a-zA-z\d]+\.[a-zA-z]+$", word))]
  summaries_final.append(filtered_words)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [13]:
for i in range(10):
  print(summaries_final[i])

['vivacious', 'lady', 'story', 'love', 'first', 'sight', 'young', 'botany', 'professor', 'named', 'peter', 'morgan', 'nightclub', 'singer', 'named', 'francey', 'also', 'comedic', 'elements', 'including', 'repeatedly', 'frustrated', 'attempts', 'newlywed', 'couple', 'find', 'moment', 'alone', 'story', 'begins', 'peter', 'sent', 'manhattan', 'retrieve', 'playboy', 'cousin', 'keith', 'james', 'ellison', 'immediately', 'falls', 'love', 'francey', 'whirlwind', 'one-day', 'courtship', 'peter', 'francey', 'get', 'married', 'keith', 'return', 'morgan', 'family', 'home', 'peter', 'teaches', 'university', 'run', 'father', 'peter', 'morgan', 'morgan', 'known', 'proud', 'overbearing', 'man', 'peter', 'afraid', 'tell', 'marriage', 'arrive', 'morgan', 'peter', 'high-society', 'fiancée', 'helen', 'initially', 'take', 'francey', 'another', 'keith', 'girlfriends', 'peter', 'decides', 'approach', 'father', 'news', 'francey', 'stays', 'women-only', 'hotel', 'peter', 'keith', 'introduce', 'new', 'botany',

## Μετατροπή σε TFIDF

Το πρώτο βήμα θα είναι λοιπόν να μετατρέψετε το corpus σε αναπαράσταση tf-idf:

Αρχικά ενημερώνουμε το corpus με βάση τα αποτελέσματα της παραπάνω ενότητας.

In [14]:
corpus_updated = corpus.copy()
for i, summary in tqdm(enumerate(summaries_final), total=5000):
  corpus_updated[i] = ' '.join(word for word in summary)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [15]:
for i in range(10):
  print(corpus_updated[i])

vivacious lady story love first sight young botany professor named peter morgan nightclub singer named francey also comedic elements including repeatedly frustrated attempts newlywed couple find moment alone story begins peter sent manhattan retrieve playboy cousin keith james ellison immediately falls love francey whirlwind one-day courtship peter francey get married keith return morgan family home peter teaches university run father peter morgan morgan known proud overbearing man peter afraid tell marriage arrive morgan peter high-society fiancée helen initially take francey another keith girlfriends peter decides approach father news francey stays women-only hotel peter keith introduce new botany student peter mentions francey father twice occasions morgan interrupts ignores son peter becomes insistent apparently ailing mother flare-up heart condition making conversation impossible third attempt peter decides announce marriage parents university student-faculty prom keith brings fra

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create sparse tf_idf representation
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus_updated)
corpus_tf_idf_plain = vectorizer.transform(corpus_updated)
print(corpus_tf_idf_plain.shape)

(5000, 48679)


Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης.

Προσοχή: ο TfidfVectorizer έχει κάποιες δυνατότητες προεπεξεργασίας παρόποιες με αυτές που αναφέραμε στην προηγούμενη ενότητα. Ό,τι προεπεξεργασία μπορείτε να κάνετε που χρειάζεται ως είσοδο μόνο το κάθε document ξεχωριστά, κάντε την στο πρώτο βήμα της προεπεξεργασίας. Αν χρειάζεται γνώση των συνολικών στατιστικών της συλλογής, κάντε την με τον TfidfVectorizer.

## Υλοποίηση του συστήματος συστάσεων

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση `content_recommender` με τρία ορίσματα: `target_movie`, `max_recommendations` και `corpus_type`. Στην `target_movie` περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, `max_recommendations` στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής: 
- για την ταινία-στόχο, θα υπολογίζετε την [ομοιότητα συνημιτόνου](https://en.wikipedia.org/wiki/Cosine_similarity) της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο `corpus_type`.
- με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (`ID`) των ταινιών. Παράδειγμα: αν η ταινία με index 1 έχει ομοιότητα συνημιτόνου με 3 ταινίες \[0.2 1 0.6\] (έχει ομοιότητα 1 με τον εαύτό της) ο ταξινομημένος αυτός πίνακας indices θα είναι \[1 2 0\].
- Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
- Για τις `max_recommendations` ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [55]:
def cosine_similarity(d1, d2):
  # use toarray() method because cosine doesn't work on sparse arrays
  return 1-sp.spatial.distance.cosine(d1.toarray(),d2.toarray())

def content_recommender(target_movie, max_recommendations, corpus_type, debug=False):
  similarity_array = []
  target_descr = corpus_type[target_movie]
  # find similarity with every other movie
  for descr in corpus_type:
    similarity_array.append(cosine_similarity(target_descr, descr))
  # sort ids of movies based on similarity
  sorted_indices = sorted(range(len(similarity_array)), key=lambda k: similarity_array[k], reverse=True)
  # return ids of movies with the greatest similarity
  if debug:
    for i in sorted_indices[1:max_recommendations+1]:
      print(similarity_array[i])
  return sorted_indices[1:max_recommendations+1]

## Βελτιστοποίηση του TfidfVectorizer

Αφού υλοποιήσετε τη συνάρτηση `content_recommender` χρησιμοποιήστε την για να βελτιστοποιήσετε την `TfidfVectorizer`. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό `max_recommendations` (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το `ID` τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την `TfidfVectorizer` για τα συγκεκριμένα `ID` ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό `max_recommendations`. Παράλληλα, όσο βελτιστοποιείτε την `TfidfVectorizer`, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών. 

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του `TfidfVectorizer` έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων. 




Για 2 τυχαίες ταινίες (έστω ID 519 και 4120) βρίσκουμε τις 3 κοντινότερες ταινίες που προτείνει ο recommender και βλέπουμε τα summaries και τα categories της κάθε πρότασης. Για ταινίες που φαίνονται να είναι σημασιολογικά κοντά στις αρχικές ταινίες σημειώνουμε το ID τους.

In [23]:
id = 519
films = content_recommender(id, 3, corpus_tf_idf_plain)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 519
Its categories are ['"Japanese Movies"']
Its summary is: Even though being able to withdraw from the Liar Game Tournament, Kanzaki Nao  decides to join Akiyama Shinichi  and nine other players for the final stage, ”The Garden of Eden game”, which offers a 5 billion yen prize. The players are informed that if all of them can work together, they can all become winners. However, the chances of winning are endangered by a mysterious player known as "Person X".
Recommended film has ID 4252
Its categories are ['"Horror",  "Slasher",  "Teen"']
Its summary is: A group of teenagers create a game where one of them is secretly a killer while the others are victims. Using prop weapons, the killer must eliminate the other players before being discovered. The teens sneak into a warehouse late at night to play the game but things turn horrific when the players begin dying for real.
Recommended film has ID 3978
Its categories are ['"Family Drama",  "Docudrama", 

Για την ταινία με ID 519 οι σημασιολογικά κοντινότερη ταινία σύμφωνα με τα categories και το summary φαίνεται να είναι μόνο η 4252.

In [58]:
id = 4120
films = content_recommender(id, 3, corpus_tf_idf_plain)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 4120
Its categories are ['"Thriller",  "Crime Fiction",  "Science Fiction",  "World cinema",  "Fantasy",  "Comedy"']
Its summary is: Coco  and his 7-month pregnant wife Pipi , have recently moved into a Buenos Aires apartment and fail to notice the increasing chaos surrounding them. Shortly, authorities quarantine the building and Coco allies himself with his well-prepared neighbour, Horacio, to defend his refrigerator and keep Pipi safe while the world outside the apartment is falling apart.
Recommended film has ID 3599
Its categories are ['"Comedy"']
Its summary is: Coco is a wealthy Sephardic Jewish businessman and an immigrant to France. After 15 years of financial success  he decides to throw an enormous bar mitzvah celebration for his son Samuel. Determined that the celebration will be the most fantastic event of the year, Coco alienates his entire family as he plans for the gala.
Recommended film has ID 4295
Its categories are ['"Horror",  "Co

Για την ταινία 4120 η πιο σωστή σύσταση είναι αυτή της ταινίας με ID 4295.

Στη συνέχεια αυστηροποιούμε τα κριτήρια του TfidfVectorizer() ώστε να μειώσουμε τις διαστάσεις του προβλήματος και να προσπαθήσουμε να κάνουμε τις συστάσεις πιο συγκεκριμένες.

In [68]:
vectorizer = TfidfVectorizer(max_df=.5, min_df=2)
vectorizer.fit(corpus_updated)
corpus_tf_idf = vectorizer.transform(corpus_updated)
print(corpus_tf_idf.shape)

(5000, 24755)


Αρχικά δοκιμάζουμε τον νέο TfidfVectorizer στην ταινία με ID 519. Αυξάνουμε το πλήθος των προτεινόμενων ταινιών που επιστρέφονται ώστε να αξιολογήσουμε την ακρίβεια του recommender.

In [71]:
id = 519
films = content_recommender(id, 15, corpus_tf_idf)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 519
Its categories are ['"Japanese Movies"']
Its summary is: Even though being able to withdraw from the Liar Game Tournament, Kanzaki Nao  decides to join Akiyama Shinichi  and nine other players for the final stage, ”The Garden of Eden game”, which offers a 5 billion yen prize. The players are informed that if all of them can work together, they can all become winners. However, the chances of winning are endangered by a mysterious player known as "Person X".
Recommended film has ID 4252
Its categories are ['"Horror",  "Slasher",  "Teen"']
Its summary is: A group of teenagers create a game where one of them is secretly a killer while the others are victims. Using prop weapons, the killer must eliminate the other players before being discovered. The teens sneak into a warehouse late at night to play the game but things turn horrific when the players begin dying for real.
Recommended film has ID 2750
Its categories are ['"Thriller",  "Science Fiction"

Παρατηρούμε πως υπάρχουν ακόμα αρκετές ταινίες που δεν σχετίζονται με την αρχική ταινία, έχουμε όμως και μερικές όπως οι 2750, 4400, 1166 2152 και 1892 οι οποίες έχουν κοντινή θεματολογία με αυτή. Επαναλαμβάνουμε για την ταινία 4120.

In [73]:
id = 4120
films = content_recommender(id, 15, corpus_tf_idf)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 4120
Its categories are ['"Thriller",  "Crime Fiction",  "Science Fiction",  "World cinema",  "Fantasy",  "Comedy"']
Its summary is: Coco  and his 7-month pregnant wife Pipi , have recently moved into a Buenos Aires apartment and fail to notice the increasing chaos surrounding them. Shortly, authorities quarantine the building and Coco allies himself with his well-prepared neighbour, Horacio, to defend his refrigerator and keep Pipi safe while the world outside the apartment is falling apart.
Recommended film has ID 3599
Its categories are ['"Comedy"']
Its summary is: Coco is a wealthy Sephardic Jewish businessman and an immigrant to France. After 15 years of financial success  he decides to throw an enormous bar mitzvah celebration for his son Samuel. Determined that the celebration will be the most fantastic event of the year, Coco alienates his entire family as he plans for the gala.
Recommended film has ID 4295
Its categories are ['"Horror",  "Co

Εδώ βλέπουμε πως η μείωση της διαστατικότητας δεν βοήθησε τον recommender αφού οι περισσότερες από τις ταινίες δεν σχετίζονται με την ταινία που δώσαμε σε αυτόν.

Περιορίζουμε ακόμα περισσότερο τις διαστάσεις του προβλήματος και ελέγχουμε όπως και πριν το αποτέλεσμα.

In [134]:
vectorizer = TfidfVectorizer(max_df=500, min_df=2, max_features=10000)
vectorizer.fit(corpus_updated)
corpus_tf_idf_2 = vectorizer.transform(corpus_updated)
print(corpus_tf_idf_2.shape)

(5000, 10000)


In [135]:
id = 519
films = content_recommender(id, 15, corpus_tf_idf_2)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 519
Its categories are ['"Japanese Movies"']
Its summary is: Even though being able to withdraw from the Liar Game Tournament, Kanzaki Nao  decides to join Akiyama Shinichi  and nine other players for the final stage, ”The Garden of Eden game”, which offers a 5 billion yen prize. The players are informed that if all of them can work together, they can all become winners. However, the chances of winning are endangered by a mysterious player known as "Person X".
Recommended film has ID 4252
Its categories are ['"Horror",  "Slasher",  "Teen"']
Its summary is: A group of teenagers create a game where one of them is secretly a killer while the others are victims. Using prop weapons, the killer must eliminate the other players before being discovered. The teens sneak into a warehouse late at night to play the game but things turn horrific when the players begin dying for real.
Recommended film has ID 2750
Its categories are ['"Thriller",  "Science Fiction"

In [136]:
id = 4120
films = content_recommender(id, 15, corpus_tf_idf_2)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 4120
Its categories are ['"Thriller",  "Crime Fiction",  "Science Fiction",  "World cinema",  "Fantasy",  "Comedy"']
Its summary is: Coco  and his 7-month pregnant wife Pipi , have recently moved into a Buenos Aires apartment and fail to notice the increasing chaos surrounding them. Shortly, authorities quarantine the building and Coco allies himself with his well-prepared neighbour, Horacio, to defend his refrigerator and keep Pipi safe while the world outside the apartment is falling apart.
Recommended film has ID 4295
Its categories are ['"Horror",  "Comedy"']
Its summary is: The film opens with a couple drunk driving across the streets of New York to their apartment. They get home and make love in front of the kitchen. When the wife walks into the kitchen, the refrigerator opens up and sucks the girl in. We then go to Steve & Eileen Batemen. They are moving to New York, and they move into the same apartment with the killer refrigerator. Steve tak

Με τα νέα χαρακτηριστικά η επίδοση του recommender φαίνεται να βελτιώθηκε. Για να έχουμε μια ολοκληρωμένη εικόνα θα επαναλάβουμε τη διαδικασία και για άλλες τυχαίες ταινίες ώστε να δούμε αν η καλή επίδοση γενικεύεται.

In [142]:
id = 3328
films = content_recommender(id, 7, corpus_tf_idf_2)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 3328
Its categories are ['"Period piece",  "Biography",  "Drama"']
Its summary is: John suffered from epileptic seizures and an autism-like developmental disorder, and the Royal Family tried to shelter him from public view as much as possible; the script shied away from presenting the Royal Family as unsympathetic to the Prince, and instead showed how much this cost them emotionally . Poliakoff explores the story of John, his relationship with his family, John's brother, Prince George, the political events going on at the time  and the love and devotion shown to him by his nanny, Charlotte Bill , played by Gina McKee.
Recommended film has ID 2781
Its categories are ['"Silent film",  "Marriage Drama",  "Drama"']
Its summary is: John Sterling  takes his young wife Lillie  to Java where he plans to invest in tea plantations. Beyond the difference in years between the two, he is neglectful, which creates reactions of frustration from his romantic young w

In [144]:
id = 2730
films = content_recommender(id, 7, corpus_tf_idf_2)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 2730
Its categories are ['"Zombie Film",  "B-movie",  "Horror",  "Slasher"']
Its summary is: The Dead Pit opens with Dr. Ramzi , a deviant who enjoys torturing his patients, being killed and buried in the basement of a mental health facility. Twenty years later, the hospital is running again and Jane Doe  arrives at the institute. Upon her arrival, a major earthquake rocks the building and unearths the now undead Dr. Ramzi and his legion of zombie patients.
Recommended film has ID 2759
Its categories are ['"Thriller",  "Science Fiction",  "Fantasy"']
Its summary is: In 1983, deep within the mysterious Arboria Institute, a beautiful girl  is held captive by a scientist, Dr. Barry Nyle ([[Michael Rogers . Her mind is controlled by a sinister technology . Speechlessly, she waits for her next session with the deranged Dr. Nyle. She escapes her cell under the watchful eye of Dr. Nyle peering through video monitors. She journeys through the darkest reaches

Αν και δεν είναι όλες οι συστάσεις ορθές, ένα ικανοποιητικό ποσοστό από αυτές φαίνεται να είναι κοντά στις αρχικές ταινίες και ο recommender μας δουλεύει αρκετά καλά. Στη συνέχεια θα προσπαθήσουμε να μειώσουμε ακόμα περισσότερο τις διαστάσεις των χαρακτηριστικών ώστε να δούμε αν η επίδοση του θα βελτιωθεί ή αν θα έχουμε περισσότερες λανθασμένες προτάσεις.

In [145]:
vectorizer = TfidfVectorizer(max_df=200, min_df=4, max_features=5000)
vectorizer.fit(corpus_updated)
corpus_tf_idf_3 = vectorizer.transform(corpus_updated)
print(corpus_tf_idf_3.shape)

(5000, 5000)


In [147]:
id = 2311
films = content_recommender(id, 7, corpus_tf_idf_3)
print("Film to find recommendations has ID {}".format(id))
print("Its categories are {}".format(categories[id]))
print("Its summary is: {}".format(summaries[id][0]))
for film in films:
  print("Recommended film has ID {}".format(film))
  print("Its categories are {}".format(categories[film]))
  print("Its summary is: {}".format(summaries[film][0]))

Film to find recommendations has ID 2311
Its categories are ['"Culture & Society",  "Indie",  "History",  "Drama",  "Comedy",  "Political cinema",  "Social issues",  "Documentary"']
Its summary is: According to Sicko, almost fifty million Americans are uninsured while the remainder, who are covered, are often victims of insurance company fraud and red tape. Interviews are conducted with people who thought they had adequate coverage but were denied care. Former employees of insurance companies describe cost-cutting initiatives that give bonuses to insurance company physicians and others to find reasons for the company to avoid meeting the cost of medically necessary treatments for policy holders, and thus increase company profitability. In Canada, Moore describes the case of Tommy Douglas, who was voted the greatest Canadian in 2004 for his contributions to the Canadian health system. Moore also interviews a microsurgeon and people waiting in the emergency room of a Canadian public hosp

Με το πολύ μικρό πλήθος χαρακτηριστικών η επίδοση του recommender φαίνεται να έχει πέσει ακόμα περισσότερο. Καταλήγουμε λοιπόν στο συμπέρασμα πως η βέλτιστη επίδοση που μπορούμε να πετύχουμε είναι κοντά σε αυτή που είχαμε με τον προηγούμενο recommender.

## Βαθιά μάθηση: δημιουργία corpora με χρήση word emmbeddings

Η προσέγγιση της κατασκευής μόνο μέσω tfidf του συστήματος συστάσεων έχει διάφορα μειονεκτήματα. Θα μας ενδιέφερε λοιπόν να δούμε αν μπορούμε να χρησιμοποιήσουμε για τις λέξεις **εμφυτεύματα (embeddings)**, δηλαδή τις πυκνές διανυσματικές αναπαραστάσεις για τις λέξεις που μας δίνει το μοντέλο **Word2Vec**

Ωστόσο, το dataset της κάθε ομάδας είναι πολύ μικρό για να εξάγουμε τα δικά μας word embeddings (και να είναι καλά). Για το λόγο αυτό θα χρησιμοποιήσουμε τη μεθοδολογία της Βαθιάς Μάθησης που είναι η **Μεταφορά Μάθησης (Transfer Learning).**.

Στη μεταφορά μάθησης ουσιαστικά μεταφέρουμε τη γνώση που έχει αποκτήσει ένα ήδη εκπαιδευμένο (και κατά κανόνα πολύ μεγάλο) σύστημα. Η μεταφορά γίνεται διαμέσου των τιμών των βαρών που έχει προσδιορίσει μετά το πέρας της εκπαίδευσης.

Στην περίπτωσή μας, δεν μας ενδιαφέρουν τόσο τα ίδια τα βάρη των μοντέλων από τα οποία θα κάνουμε μεταφορά μάθησης. Κάτι τέτοιο θα μας ενδιέφερε αν π.χ. θέλαμε να συνεχίσουμε την εκπαίδευση στα δικά μας κείμενα. Μας ενδιαφέρουν όμως τα ίδια τα εμφυτεύματα, δηλαδή τα embeddings (διανύσματα διαστάσεων $m$) που έχει μάθει το νευρωνικό για το λεξιλόγιο του (vocabulary). To vocabulary σε τέτοια μεγάλα νευρωνικά θα είναι πιθανότατα υπερσύνολο του δικού μας.

### Μεταφορά μάθησης εμφυτευμάτων



#### Εμφυτεύματα του Gensim-data
Το Gensim περιλαμβάνει αρκετά προεκπαιδευμένα μοντέλα εμφυτευμάτων Word2Vec. Με το επόμενο κελί παίρνουμε τη λίστα τους.

In [ ]:
!pip install -U gensim
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

Τα μοντέλα αυτά βρίσκονται στο [αποθετήριο Gensim-data](https://github.com/RaRe-Technologies/gensim-data) όπου μπορείτε να βρείτε και την τεκμηρίωσή τους. Η φόρτωση των μοντέλων αυτών γίνεται με τη συνάρτηση `gensim.downloader.load`.

#### Άλλα εμφυτεύμαατα
Μπορείτε να βρείτε προεκπαιδευμένα εμφυτεύματα και από πηγές εκτός του Gensim. Για παράδειγμα:

- [Google News dataset](https://code.google.com/archive/p/word2vec/). Πρόκειται για προ-εκπαιδευμένα διανύσματα που έχουν εκπαιδευτεί σε μέρος του συνόλου δεδομένων Google News (περίπου 100 δισεκατομμύρια λέξεις). Το μοντέλο περιέχει διανύσματα 300 διαστάσεων για 3 εκατομμύρια λέξεις και φράσεις.
- [Amazon BlazingText](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). Το BlazingText δεν είναι μόνο προεκπαιδευμένα εμφυτεύματα αλλα και βελτιστοποιημένες υλοποιήσεις των αλγορίθμων Word2vec για την επεξεργασία κειμένου. Προυπόθεση είναι να δουλέψει κανείς στο SageMaker.

Οι διαδικασίες φόρτωσης embeddings από εξωτερικά δεδομένα μπορεί να είναι ελαφρά διαφορετικές από αυτή του Gensim.



#### Παρατηρήσεις

*   Επαναλαμβάνουμε ότι στην εργασία αυτή δεν μας ενδιαφέρουν τα ίδια τα μοντέλα αλλά το να μπορούμε για μία λέξη του λεξιλογίου μας να μπορούμε να βρούμε το embedding (διάνυσμα) που της αντιστοιχεί στο εκάστοτε προεκπαιδευμένο μοντέλο. 

*   Επίσης, δεν θα χρησιμοποιήσουμε την `Phrases` για να βρούμε bigrams στο dataset μας όπως θα ήταν το ορθότερο, καθώς αυτό θα απαιτούσε την συνέχιση της εκπαίδευσης του μοντέλου σε νέο λεξιλόγιο με πολύ λίγα νέα δεδομένα.


 ### Δημιουργία corpora βασισμένων στα εμφυτεύματα

Για να μπορέσουμε να ενσωματώσουμε τη γνώση που υπάρχει στα προεκπαιδευμένα εμφυτεύματα στο δικό μας corpus θα προχωρήσουμε όπως περιγράφεται ακολούθως.

Για κάθε περιγραφή ταινίας $d$, η οποία αποτελείται από τις $N_d$ λέξεις $w_i$, το  $tfidf$ της κάθε λέξης $w_i$ δίνεται από τη σχέση:

$$ tfidf(w_i) = tf(w_i,d) \cdot idf(w_i)$$

Ταυτόχρονα, σε κάθε λέξη $w_i$ αντιστοιχεί ένα διάνυσμα $W2V(w_i)$ από το μοντέλο εμφυτευμάτων που έχουμε εισάγει. Τα διανύσματα εμφυτευμάτων $W2V$ θα έχουν διάσταση $m$, ανάλογα το μοντέλο. 

Για κάθε ταινία d, μπορούμε να δημιουργήσουμε μια διανυσματική αναπαράσταση $W2V(d)$ διαστάσεων $m$ χρησιμοποιώντας το $tfidf(w_i)$ ως συντελεστή βαρύτητας για κάθε εμφύτευμα $W2V(w_i)$:

$$ W2V(d) = \frac{tfidf(w_1)\cdot W2V(w_i) + tfidf(w_2)\cdot W2V(w_2) + \dotsc  + tfidf(w_{N_{d}})\cdot W2V(w_{N_{d}})}{tfidf(w_1)+tfidf(w_2)+ \dotsc + tfidf(w_{N_{d}})}$$


#### build_tfw2v

Υλοποιήστε μια συνάρτηση `build_tfw2v` με ορίσματα:
- `corpus` που θα είναι το προεπεξεργασμένο dataset σας,
- `vectors` που θα είναι το μοντέλο που θα σας δίνει τα διανύσματα των εμφυτεύσεων vectors, και 
- `embeddings_size` που θα είναι η διάσταση των εμφυτευμάτων $m$.

H συνάρτηση αυτή θα επιστρέφει ένα νέο corpus που θα είναι ένας πίνακας 5000 (όσες οι ταινίες σας) x $m$ (το η διάσταση των εμφυτευμάτων). Ανάλογα ποιο μοντέλο χρησιμποιείτε για transfer learning ο πίνακας αυτός θα είναι διαφορετικός.

Μπορείτε πλεόν να καλείτε την `content_recommender` με διαφορετικά corpora στο όρισμα `corpus_type`. Σημειώστε ότι στο TFidfVectorizer χρησιμοποιουμε τη σειριακή μορφή των numpy arrays και ίσως σας χρησιμεύσει η `sparse.csr_matrix()` από την Scipy.

## Ανάλυση αποτελεσμάτων

### Σύστημα συστάσεων βασισμένο μόνο στο tfidf

- Σε markdown περιγράψτε τι προεπεξεργασία κάνετε στα κείμενα και γιατί.

- Περιγράψτε πως προχωρήσατε στις επιλογές σας για τη βελτιστοποίηση της `TfidfVectorizer`. 

- [Cherry-picking:](https://www.wikiwand.com/en/Cherry_picking) Δώσετε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν καλά αποτελέσματα μέχρι `max_recommendations` (τουλάχιστον 5) και σχολιάστε.

- [Nir-picking:](https://www.merriam-webster.com/dictionary/nit-picking) Δώστε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν κακά αποτελέσματα και σχολιάστε.

- Ποια είναι συνολικά τα πλεονεκτήματα και μειονεκτήματα ενός recommender βασισμένου στο tfidf;

### Σύγκριση και σχολιασμός με recommenders βασισμένων στο Word2Vec

- Υλoποιήστε recommenders που βασίζονται σε μεταφορά μάθησης και εμφυτεύματα. Χρησιμοποιήστε παραδείγματα για να υποδείξετε δυνατά και αδύναμα σημεία τους.

- Μπορείτε να σχολιάσετε τα recommenders που βασίζονται στο Word2Vec σε σχέση με το απλό μοντέλο tfidf, εξετάζοντας τις συστάσεις για ίδια ID.

- Μπορείτε επίσης να εξετάσετε συγκριτικά τα Word2Vec recommenders μεταξύ τους και πάλι βασιζόμενοι σε παραδείγματα.

- Οι παρατηρήσεις σας θα βασίζονται στην ανάλυση των ποιοτικών χαρακτηριστικών που είναι η σειρά και το σύνολο των συστάσεων. Ωστόσο, μπορείτε να συμπεριλάβετε και ποσοτικά χαρακτηριστικά όπως τους χρονους loading και συγκρότησης του corpus αλλά και της διαστατικότητας $m$.

Χρησιμοποιήστε όποια μορφή reporting κρίνετε καταλληλότερη: κείμενο, πίνακες, διαγράμματα.


# Εφαρμογή 2.  Τοπολογική και σημασιολογική απεικόνιση της ταινιών με χρήση SOM
<img src="https://i.imgur.com/Z4FdurD.jpg" width="60%">

## Δημιουργία dataset
Στη δεύτερη εφαρμογή θα βασιστούμε στις τοπολογικές ιδιότητες των Self Organizing Maps (SOM) για να φτιάξουμε ενά χάρτη (grid) δύο διαστάσεων όπου θα απεικονίζονται όλες οι ταινίες της συλλογής της ομάδας με τρόπο χωρικά συνεκτικό ως προς το περιεχόμενο και κυρίως το είδος τους (ο παραπάνω χάρτης είναι ενδεικτικός, δεν αντιστοιχεί στο dataset μας). 

Διαλέξτε για την αναπαράσταση των documents αυτήν που πιστεύετε απέδωσε καλύτερα στο πρώτα σκέλος της άσκησης. Έστω ότι αυτή είναι η `my_best_corpus`.

Η έτοιμη συνάρτηση `build_final_set` θα ενώσει την αναπαράσταση που θα της δώσετε ως όρισμα `mycorpus` με τις binarized κατηγορίες `catbins` των ταινιών ως επιπλέον κολόνες (χαρακτηριστικά). Συνεπώς, κάθε ταινία αναπαρίσταται στο Vector Space Model από τα χαρακτηριστικά της αναπαράστασης `mycorpus` και τις κατηγορίες της.

Τέλος, η συνάρτηση δέχεται ένα ορισμα για το πόσες ταινίες να επιστρέψει, με default τιμή όλες τις ταινίες (5000). Αυτό είναι χρήσιμο για να μπορείτε αν θέλετε να φτιάχνετε μικρότερα σύνολα δεδομένων ώστε να εκπαιδεύεται ταχύτερα το SOM. 

Θα τρέχουμε τη συνάρτηση με `final_set = build_final_set(my_best_corpus)`.

In [ ]:
def build_final_set(mycorpus, doc_limit = 5000, tf_idf_only=False):
    # convert sparse tf_idf to dense tf_idf representation
    dense_tf_idf = mycorpus.toarray()[0:doc_limit,:]
    if tf_idf_only:
        # use only tf_idf
        final_set = dense_tf_idf
    else:
        # append the binary categories features horizontaly to the (dense) tf_idf features
        final_set = np.hstack((dense_tf_idf, catbins[0:doc_limit,:]))
    # η somoclu θέλει δεδομ΄ένα σε float32
    return np.array(final_set, dtype=np.float32)

Στο επόμενο κελί, τυπώνουμε τις διαστάσεις του τελικού dataset μας. **Χωρίς βελτιστοποίηση του TFIDF** θα έχουμε περίπου 50.000 χαρακτηριστικά και ο θα είναι ανέφικτο να προχωρήσουμε στην εκπαίδευση του SOM.

In [ ]:
final_set.shape

## Εκπαίδευση χάρτη SOM

Θα δουλέψουμε με τη βιβλιοθήκη SOM ["Somoclu"](http://somoclu.readthedocs.io/en/stable/index.html). Εισάγουμε τις somoclu και matplotlib και λέμε στη matplotlib να τυπώνει εντός του notebook (κι όχι σε pop up window).

In [ ]:
# install somoclu
!pip install --upgrade somoclu
# import sompoclu, matplotlib
import somoclu
import matplotlib
# we will plot inside the notebook and not in separate window
%matplotlib inline

Καταρχάς διαβάστε το [function reference](http://somoclu.readthedocs.io/en/stable/reference.html) του somoclu. Θα δoυλέψουμε με χάρτη τύπου planar, παραλληλόγραμμου σχήματος νευρώνων με τυχαία αρχικοποίηση (όλα αυτά είναι default). Μπορείτε να δοκιμάσετε διάφορα μεγέθη χάρτη ωστόσο όσο ο αριθμός των νευρώνων μεγαλώνει, μεγαλώνει και ο χρόνος εκπαίδευσης. Για το training δεν χρειάζεται να ξεπεράσετε τα 100 epochs. Σε γενικές γραμμές μπορούμε να βασιστούμε στις default παραμέτρους μέχρι να έχουμε τη δυνατότητα να οπτικοποιήσουμε και να αναλύσουμε ποιοτικά τα αποτελέσματα. Ξεκινήστε με ένα χάρτη 10 x 10, 100 epochs training και ένα υποσύνολο των ταινιών (π.χ. 2000). Χρησιμοποιήστε την `time` για να έχετε μια εικόνα των χρόνων εκπαίδευσης. 


## Best matching units

Μετά από κάθε εκπαίδευση αποθηκεύστε σε μια μεταβλητή τα best matching units (bmus) για κάθε ταινία. Τα bmus μας δείχνουν σε ποιο νευρώνα ανήκει η κάθε ταινία. **Προσοχή: η σύμβαση των συντεταγμένων των νευρώνων στη Somoclu είναι (στήλη, γραμμή) δηλαδή το ανάποδο από την Python**. Με χρήση της [np.unique](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html) (μια πολύ χρήσιμη συνάρτηση στην άσκηση) αποθηκεύστε τα μοναδικά best matching units και τους δείκτες τους (indices) προς τις ταινίες. 

Σημειώστε ότι μπορεί να έχετε λιγότερα μοναδικά bmus από αριθμό νευρώνων γιατί μπορεί σε κάποιους νευρώνες να μην έχουν ανατεθεί ταινίες. Ως αριθμό νευρώνα θα θεωρήσουμε τον αριθμό γραμμής στον πίνακα μοναδικών bmus.



## Ομαδοποίηση (clustering)

Τυπικά, η ομαδοποίηση σε ένα χάρτη SOM προκύπτει από το unified distance matrix (U-matrix): για κάθε κόμβο υπολογίζεται η μέση απόστασή του από τους γειτονικούς κόμβους. Εάν χρησιμοποιηθεί μπλε χρώμα στις περιοχές του χάρτη όπου η τιμή αυτή είναι χαμηλή (μικρή απόσταση) και κόκκινο εκεί που η τιμή είναι υψηλή (μεγάλη απόσταση), τότε μπορούμε να πούμε ότι οι μπλε περιοχές αποτελούν clusters και οι κόκκινες αποτελούν σύνορα μεταξύ clusters.

To somoclu δίνει την επιπρόσθετη δυνατότητα να κάνουμε ομαδοποίηση των νευρώνων χρησιμοποιώντας οποιονδήποτε αλγόριθμο ομαδοποίησης του scikit-learn. Στην άσκηση θα χρησιμοποιήσουμε τον k-Means. Για τον αρχικό σας χάρτη δοκιμάστε ένα k=20 ή 25. Οι δύο προσεγγίσεις ομαδοποίησης είναι διαφορετικές, οπότε περιμένουμε τα αποτελέσματα να είναι κοντά αλλά όχι τα ίδια.



## Αποθήκευση του SOM

Επειδή η αρχικοποίηση του SOM γίνεται τυχαία και το clustering είναι και αυτό στοχαστική διαδικασία, οι θέσεις και οι ετικέτες των νευρώνων και των clusters θα είναι διαφορετικές κάθε φορά που τρέχετε τον χάρτη, ακόμα και με τις ίδιες παραμέτρους. Για να αποθηκεύσετε ένα συγκεκριμένο som και clustering χρησιμοποιήστε και πάλι την `joblib`. Μετά την ανάκληση ενός SOM θυμηθείτε να ακολουθήσετε τη διαδικασία για τα bmus.



## Οπτικοποίηση U-matrix, clustering και μέγεθος clusters

Για την εκτύπωση του U-matrix χρησιμοποιήστε τη `view_umatrix` με ορίσματα `bestmatches=True` και `figsize=(15, 15)` ή `figsize=(20, 20)`. Τα διαφορετικά χρώματα που εμφανίζονται στους κόμβους αντιπροσωπεύουν τα διαφορετικά clusters που προκύπτουν από τον k-Means. Μπορείτε να εμφανίσετε τη λεζάντα του U-matrix με το όρισμα `colorbar`. Μην τυπώνετε τις ετικέτες (labels) των δειγμάτων, είναι πολύ μεγάλος ο αριθμός τους.

Για μια δεύτερη πιο ξεκάθαρη οπτικοποίηση του clustering τυπώστε απευθείας τη μεταβλητή `clusters`.

Τέλος, χρησιμοποιώντας πάλι την `np.unique` (με διαφορετικό όρισμα) και την `np.argsort` (υπάρχουν και άλλοι τρόποι υλοποίησης) εκτυπώστε τις ετικέτες των clusters (αριθμοί από 0 έως k-1) και τον αριθμό των νευρώνων σε κάθε cluster, με φθίνουσα ή αύξουσα σειρά ως προς τον αριθμό των νευρώνων. Ουσιαστικά είναι ένα εργαλείο για να βρίσκετε εύκολα τα μεγάλα και μικρά clusters. 

Ακολουθεί ένα μη βελτιστοποιημένο παράδειγμα για τις τρεις προηγούμενες εξόδους:

<img src="https://image.ibb.co/i0tsfR/umatrix_s.jpg" width="35%">
<img src="https://image.ibb.co/nLgHEm/clusters.png" width="35%">




## Σημασιολογική ερμηνεία των clusters

Προκειμένου να μελετήσουμε τις τοπολογικές ιδιότητες του SOM και το αν έχουν ενσωματώσει σημασιολογική πληροφορία για τις ταινίες διαμέσου της διανυσματικής αναπαράστασης του tf-idf, των εμφυτευμάτων και των κατηγοριών, χρειαζόμαστε ένα κριτήριο ποιοτικής επισκόπησης των clusters. 

Θα υλοποιήσουμε το εξής κριτήριο: Λαμβάνουμε όρισμα έναν αριθμό (ετικέτα) cluster. Για το cluster αυτό βρίσκουμε όλους τους νευρώνες που του έχουν ανατεθεί από τον k-Means. Για όλους τους νευρώνες αυτούς βρίσκουμε όλες τις ταινίες που τους έχουν ανατεθεί (για τις οποίες αποτελούν bmus). Για όλες αυτές τις ταινίες τυπώνουμε ταξινομημένη τη συνολική στατιστική όλων των ειδών (κατηγοριών) και τις συχνότητές τους. Αν το cluster διαθέτει καλή συνοχή και εξειδίκευση, θα πρέπει κάποιες κατηγορίες να έχουν σαφώς μεγαλύτερη συχνότητα από τις υπόλοιπες. Θα μπορούμε τότε να αναθέσουμε αυτήν/ές την/τις κατηγορία/ες ως ετικέτες κινηματογραφικού είδους στο cluster.

Μπορείτε να υλοποιήσετε τη συνάρτηση αυτή όπως θέλετε. Μια πιθανή διαδικασία θα μπορούσε να είναι η ακόλουθη:

1. Ορίζουμε συνάρτηση `print_categories_stats` που δέχεται ως είσοδο λίστα με ids ταινιών. Δημιουργούμε μια κενή λίστα συνολικών κατηγοριών. Στη συνέχεια, για κάθε ταινία επεξεργαζόμαστε το string `categories` ως εξής: δημιουργούμε μια λίστα διαχωρίζοντας το string κατάλληλα με την `split` και αφαιρούμε τα whitespaces μεταξύ ετικετών με την `strip`. Προσθέτουμε τη λίστα αυτή στη συνολική λίστα κατηγοριών με την `extend`. Τέλος χρησιμοποιούμε πάλι την `np.unique` για να μετρήσουμε συχνότητα μοναδικών ετικετών κατηγοριών και ταξινομούμε με την `np.argsort`. Τυπώνουμε τις κατηγορίες και τις συχνότητες εμφάνισης ταξινομημένα. Χρήσιμες μπορεί να σας φανούν και οι `np.ravel`, `np.nditer`, `np.array2string` και `zip`.

2. Ορίζουμε τη βασική μας συνάρτηση `print_cluster_neurons_movies_report` που δέχεται ως όρισμα τον αριθμό ενός cluster. Με τη χρήση της `np.where` μπορούμε να βρούμε τις συντεταγμένες των bmus που αντιστοιχούν στο cluster και με την `column_stack` να φτιάξουμε έναν πίνακα bmus για το cluster. Προσοχή στη σειρά (στήλη - σειρά) στον πίνακα bmus. Για κάθε bmu αυτού του πίνακα ελέγχουμε αν υπάρχει στον πίνακα μοναδικών bmus που έχουμε υπολογίσει στην αρχή συνολικά και αν ναι προσθέτουμε το αντίστοιχο index του νευρώνα σε μια λίστα. Χρήσιμες μπορεί να είναι και οι `np.rollaxis`, `np.append`, `np.asscalar`. Επίσης πιθανώς να πρέπει να υλοποιήσετε ένα κριτήριο ομοιότητας μεταξύ ενός bmu και ενός μοναδικού bmu από τον αρχικό πίνακα bmus.

3. Υλοποιούμε μια βοηθητική συνάρτηση `neuron_movies_report`. Λαμβάνει ένα σύνολο νευρώνων από την `print_cluster_neurons_movies_report` και μέσω της `indices` φτιάχνει μια λίστα με το σύνολο ταινιών που ανήκουν σε αυτούς τους νευρώνες. Στο τέλος καλεί με αυτή τη λίστα την `print_categories_stats` που τυπώνει τις στατιστικές των κατηγοριών.

Μπορείτε βέβαια να προσθέσετε οποιαδήποτε επιπλέον έξοδο σας βοηθάει. Μια χρήσιμη έξοδος είναι πόσοι νευρώνες ανήκουν στο cluster και σε πόσους και ποιους από αυτούς έχουν ανατεθεί ταινίες.

Θα επιτελούμε τη σημασιολογική ερμηνεία του χάρτη καλώντας την `print_cluster_neurons_movies_report` με τον αριθμός ενός cluster που μας ενδιαφέρει. 

Παράδειγμα εξόδου για ένα cluster (μη βελτιστοποιημένος χάρτης, ωστόσο βλέπετε ότι οι μεγάλες κατηγορίες έχουν σημασιολογική  συνάφεια):

```
Overall Cluster Genres stats:  
[('"Horror"', 86), ('"Science Fiction"', 24), ('"B-movie"', 16), ('"Monster movie"', 10), ('"Creature Film"', 10), ('"Indie"', 9), ('"Zombie Film"', 9), ('"Slasher"', 8), ('"World cinema"', 8), ('"Sci-Fi Horror"', 7), ('"Natural horror films"', 6), ('"Supernatural"', 6), ('"Thriller"', 6), ('"Cult"', 5), ('"Black-and-white"', 5), ('"Japanese Movies"', 4), ('"Short Film"', 3), ('"Drama"', 3), ('"Psychological thriller"', 3), ('"Crime Fiction"', 3), ('"Monster"', 3), ('"Comedy"', 2), ('"Western"', 2), ('"Horror Comedy"', 2), ('"Archaeology"', 2), ('"Alien Film"', 2), ('"Teen"', 2), ('"Mystery"', 2), ('"Adventure"', 2), ('"Comedy film"', 2), ('"Combat Films"', 1), ('"Chinese Movies"', 1), ('"Action/Adventure"', 1), ('"Gothic Film"', 1), ('"Costume drama"', 1), ('"Disaster"', 1), ('"Docudrama"', 1), ('"Film adaptation"', 1), ('"Film noir"', 1), ('"Parody"', 1), ('"Period piece"', 1), ('"Action"', 1)]```
   


## Tips για το SOM και το clustering

- Για την ομαδοποίηση ένα U-matrix καλό είναι να εμφανίζει και μπλε-πράσινες περιοχές (clusters) και κόκκινες περιοχές (ορίων). Παρατηρήστε ποια σχέση υπάρχει μεταξύ αριθμού ταινιών στο final set, μεγέθους grid και ποιότητας U-matrix.
- Για το k του k-Means προσπαθήστε να προσεγγίζει σχετικά τα clusters του U-matrix (όπως είπαμε είναι διαφορετικοί μέθοδοι clustering). Μικρός αριθμός k δεν θα σέβεται τα όρια. Μεγάλος αριθμός θα δημιουργεί υπο-clusters εντός των clusters που φαίνονται στο U-matrix. Το τελευταίο δεν είναι απαραίτητα κακό, αλλά μεγαλώνει τον αριθμό clusters που πρέπει να αναλυθούν σημασιολογικά.
- Σε μικρούς χάρτες και με μικρά final sets δοκιμάστε διαφορετικές παραμέτρους για την εκπαίδευση του SOM. Σημειώστε τυχόν παραμέτρους που επηρεάζουν την ποιότητα του clustering για το dataset σας ώστε να τις εφαρμόσετε στους μεγάλους χάρτες.
- Κάποια τοπολογικά χαρακτηριστικά εμφανίζονται ήδη σε μικρούς χάρτες. Κάποια άλλα χρειάζονται μεγαλύτερους χάρτες. Δοκιμάστε μεγέθη 20x20, 25x25 ή και 30x30 και αντίστοιχη προσαρμογή των k. Όσο μεγαλώνουν οι χάρτες, μεγαλώνει η ανάλυση του χάρτη αλλά μεγαλώνει και ο αριθμός clusters που πρέπει να αναλυθούν.




## Ανάλυση τοπολογικών ιδιοτήτων χάρτη SOM

Μετά το πέρας της εκπαίδευσης και του clustering θα έχετε ένα χάρτη με τοπολογικές ιδιότητες ως προς τα είδη των ταίνιών της συλλογής σας, κάτι αντίστοιχο με την εικόνα στην αρχή της Εφαρμογής 2 αυτού του notebook. Η συγκεκριμένη εικόνα είναι μόνο για εικονογράφιση, δεν είναι χάρτης SOM καιδεν έχει καμία σχέση με τη συλλογή δεδομένων και τις κατηγορίες μας.

Για τον τελικό χάρτη SOM που θα παράξετε για τη συλλογή σας, αναλύστε σε markdown με συγκεκριμένη αναφορά σε αριθμούς clusters και τη σημασιολογική ερμηνεία τους τις εξής τρεις τοπολογικές ιδιότητες του SOM: 

1. Δεδομένα που έχουν μεγαλύτερη πυκνότητα πιθανότητας στο χώρο εισόδου τείνουν να απεικονίζονται με περισσότερους νευρώνες στο χώρο μειωμένης διαστατικότητας. Δώστε παραδείγματα από συχνές και λιγότερο συχνές κατηγορίες ταινιών. Χρησιμοποιήστε τις στατιστικές των κατηγοριών στη συλλογή σας και τον αριθμό κόμβων που χαρακτηρίζουν.
2. Μακρινά πρότυπα εισόδου τείνουν να απεικονίζονται απομακρυσμένα στο χάρτη. Υπάρχουν χαρακτηριστικές κατηγορίες ταινιών που ήδη από μικρούς χάρτες τείνουν να τοποθετούνται σε διαφορετικά ή απομονωμένα σημεία του χάρτη.
3. Κοντινά πρότυπα εισόδου τείνουν να απεικονίζονται κοντά στο χάρτη. Σε μεγάλους χάρτες εντοπίστε είδη ταινιών και κοντινά τους υποείδη.

Προφανώς τοποθέτηση σε 2 διαστάσεις που να σέβεται μια απόλυτη τοπολογία δεν είναι εφικτή, αφενός γιατί δεν υπάρχει κάποια απόλυτη εξ ορισμού για τα κινηματογραφικά είδη ακόμα και σε πολλές διαστάσεις, αφετέρου γιατί πραγματοποιούμε μείωση διαστατικότητας.

Εντοπίστε μεγάλα clusters και μικρά clusters που δεν έχουν σαφή χαρακτηριστικά. Εντοπίστε clusters συγκεκριμένων ειδών που μοιάζουν να μην έχουν τοπολογική συνάφεια με γύρω περιοχές. Προτείνετε πιθανές ερμηνείες.


Τέλος, εντοπίστε clusters που έχουν κατά την άποψή σας ιδιαίτερο ενδιαφέρον στη συλλογή της ομάδας σας (data exploration / discovery value) και σχολιάστε.



# Τελική παράδοση άσκησης

- Θα παραδώσετε στο helios το παρόν notebook επεξεργασμένο ή ένα ή δύο νέα zipαρισμένα με τις απαντήσεις σας για τα ζητούμενα και των δύο εφαρμογών. 
- Θυμηθείτε ότι η ανάλυση του χάρτη στο markdown με αναφορά σε αριθμούς clusters πρέπει να αναφέρεται στον τελικό χάρτη με τα κελιά ορατά που θα παραδώσετε αλλιώς ο χάρτης που θα προκύψει θα είναι διαφορετικός και τα labels των clusters δεν θα αντιστοιχούν στην ανάλυσή σας. 
- Μην ξεχάσετε στην αρχή ένα κελί markdown με **τα στοιχεία της ομάδας σας**.

<table>
  <tr><td align="center">
    <font size="4">Παρακαλούμε διατρέξτε βήμα-βήμα το notebook για να μην ξεχάσετε παραδοτέα</font>
</td>
  </tr>
</table>